In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 85.2 MB/s 


In [1]:
import pickle
import pandas as pd
import numpy as np

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [4]:
with open('/content/drive/MyDrive/프로젝트/data/PKDFFinal.p', 'rb') as f:
    df = pickle.load(f)
df

,title,artist,lyric,genre
0,내 손을 잡아,아이유,"[느낌이 오잖아, 떨리고 있잖아, 언제까지 눈치만 볼 거니, 네 맘을 말해봐, 딴청...",balad
1,너의 모든 순간 (Original),성시경,"[이윽고 내가 한눈에, 너를 알아봤을 때, 모든 건 분명 달라지고 있었어, 내 세상...",balad
2,Timeless,SG 워너비,"[어쩜 살아가다 보면, 한 번은 날 찾을지 몰라, 난 그 기대 하나로, 오늘도 힘겹...",balad
3,바람이 불었으면 좋겠어,길구봉구,"[웃는 모습이 너무 아름다운, 그런 너를 기억하면서, 괜찮아 넌 잘할거라 말하던, ...",balad
4,봄 사랑 벚꽃 말고,HIGH4 (하이포),"[길었던 겨우내 줄곧 품이 좀 남는, 밤색 코트, 그 속에 나를 쏙 감추고 걸음을,...",balad
...,...,...,...,...
14997,NaN,NaN,"[다시는 나의 이러한 맘뿐은, 때가 되면 그림자 같이, 당신한테로 가오리다]",NaN
14998,NaN,NaN,"[그립다 말을 할까 하니, 그리워]",NaN
14999,NaN,NaN,"[그냥 갈까 그래도 다시, 한번]",NaN
15000,NaN,NaN,"[저 산에도 까마귀 들에, 까마귀 서산에는 해 진다고, 지저귑니다]",NaN


In [ ]:
df

In [ ]:
with open('/content/drive/MyDrive/프로젝트/모델/tokenizer6.p', 'rb') as f:
    tokenizer = pickle.load(f)

In [15]:
m = load_model('/content/drive/MyDrive/프로젝트/모델/LSTM8p.h5')

In [5]:
from nltk import ConditionalFreqDist, ConditionalProbDist, MLEProbDist
from nltk.util import ngrams

In [6]:
stopwords = ['가', '예', '이', '리가', '하', '오', '지', '도', 
             '마치', '와', '우', '거야', '그', '끼', '번','으로','로'
             '에서', '걸','해','고','얄라리','줘요내']

In [7]:
x_data = []
for i in range(len(df)):
    words = ' '.join(df.iloc[i,2]).split()

    x_data.append([w for w in words if w not in stopwords])

In [ ]:
x_data[0]

In [8]:
data_l = []
for lyric in x_data:
    data = ngrams(lyric, 2, pad_left=True, pad_right = True, left_pad_symbol='SS', right_pad_symbol = 'SE')
    data_l += [l for l in data]

In [ ]:
data_l[1]

('느낌이', '오잖아')

In [9]:
cfd = ConditionalFreqDist(data_l)

In [10]:
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [ ]:
cpd

<ConditionalProbDist with 122362 conditions>

In [ ]:
def make_nouns(sentence, okt = okt):
    word = sentence.split()
    p = okt.pos(sentence)
    words = []
    for i in range(len(p)):
        if p[i][1] ==  'Noun':
            words.append(word[i])
    return words

In [16]:
# 단어 여러개 넣어서 문맥이 있어보이는 척해보기
def sentence_generation(string, n, cpd = cpd, model = m, tokenizer = tokenizer): # 현재 단어, 반복할 횟수, 모델, 토크나이저 
    words = make_nouns(string)
    print(words)
    words_len = len(words)

    sentence = ''
    for i in range(words_len):
        # sentence = sentence + words[i]
        try:
            current_word = cpd[words[i]].generate()
        except:
            current_word = cpd['내'].generate() # 없는 단어라면 가장 많이 등장하는 '내'에서 generate
        
        sentence = sentence + ' ' + current_word
 
        # n번 반복
        for _ in range(n):
            encoded = tokenizer.texts_to_sequences([current_word])[0]
            encoded = pad_sequences([encoded], maxlen=10-1, padding='pre')

            # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
            result = model.predict(encoded, verbose=0)
            result = np.argmax(result, axis=1)

            for word, index in tokenizer.word_index.items(): 
                # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                if index == result:
                    break

            # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
            current_word = current_word + ' '  + word

            # 예측 단어를 문장에 저장    
            sentence = sentence + ' ' + word
        
        sentence += '\n'
        # sentence = init_word + ' ' + sentence

    return sentence

In [12]:
x_data = []
for i in range(len(df)):
    x = df.iloc[i,2]
    
    for line in x:
        xx = []
        for w in line.split():
            if w not in stopwords:
                xx.append(w)
        x_data.append(' '.join(xx))
x_data[0]

'느낌이 오잖아'

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_data)

In [17]:
def sentence_generation(current_word, n, model = m, cpd = cpd, tokenizer = tokenizer): # 현재 단어, 반복할 횟수, 모델, 토크나이저 
    words = current_word.split()
    words_len = len(words)

    sentence = ''
    for i in range(words_len):
        # sentence = sentence + words[i]

        try:
            current_word = cpd[words[i]].generate()
        except:
            current_word = cpd['내'].generate() # 없는 단어라면 가장 많이 등장하는 '내'에서 generate
        
        sentence = sentence + ' ' + current_word

        # n번 반복
        for _ in range(n-1):
            encoded = tokenizer.texts_to_sequences([current_word])[0]
            encoded = pad_sequences([encoded], maxlen=6-1, padding='pre')

            # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
            result = model.predict(encoded, verbose=0)
            result = np.argmax(result, axis=1)

            for word, index in tokenizer.word_index.items(): 
                # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                if index == result:
                    break

            # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
            current_word = current_word + ' '  + word

            # 예측 단어를 문장에 저장    
            sentence = sentence + ' ' + word
        
        sentence += '\n'
        # sentence = init_word + ' ' + sentence

    return sentence.strip()

In [ ]:
word = input('문장 혹은 단어를 입력하세요.\n')
print()
n = int(input('출력할 문장의 길이(n)를 정해주세요.(2 < n < 10)\n'))
print()
print(sentence_generation(word, n))

문장 혹은 단어를 입력하세요.
친구의 사랑

출력할 문장의 길이(n)를 정해주세요.(2 < n < 10)
7

질투마저 알어질가도 간 햇살 속에서 건네야 싶어
 만날 때 아름다운 사랑보다는 하루를 해봤니 술을


In [39]:
encoded = tokenizer.texts_to_sequences(['친구'])[0]
encoded = pad_sequences([encoded], maxlen=6-1, padding='pre')

In [40]:
result = m.predict(encoded, verbose=0)
result = np.argmax(result, axis=1)

tokenizer.index_word[result[0]]

'아들의'

In [41]:
for word, index in tokenizer.word_index.items(): 
                # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
    if index == result:
        break

In [42]:
word

'아들의'

In [ ]:
import re
re.

In [38]:
def sentence_generation(input_word, n, model = m, cpd = cpd, tokenizer = tokenizer): # 현재 단어, 반복할 횟수, 모델, 토크나이저 
    import re
    input_word_ = ''

    for w in input_word.split():
        w = re.sub('[^가-힣]', '', w) # 단어가 될 수 있는 글자만 남김
        input_word_ = input_word_ + ' ' + w

    sentence = ''

    try:
        words = input_word_.strip().split()[-1]  # 외쪽 공백 지우기고 맨 마지막 단어 꺼내기
    except: # 모든 문자가 가-힣이 아니라 아예 빈 문자열이 된다면
        sentence = '죄송합니다. 문장(단어)을 이해하지 못하겠어요. 대신 특정 문장을 소개해드릴게요.\n'
    
    try:
        current_word = cpd[words].generate()
    except:
        current_word = cpd['내'].generate() # 없는 단어라면 가장 많이 등장하는 '내'에서 generate
        sentence = '죄송합니다. 문장(단어)을 이해하지 못하겠어요. 대신 특정 문장을 소개해드릴게요.\n'

    current_word = input_word_ + ' ' + current_word

        # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=6-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        word = tokenizer.index_word[result[0]]

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장    
        sentence = sentence + ' ' + word

    return sentence.strip()

In [50]:
word = input('문장 혹은 단어를 입력하세요.\n')
print()
n = int(input('출력할 문장의 길이(n)를 정해주세요.(2 < n < 15)\n'))
print()
print(sentence_generation(word, n))

문장 혹은 단어를 입력하세요.
사랑하는 사람이 있습니다

출력할 문장의 길이(n)를 정해주세요.(2 < n < 15)
10

햇살 속 그대와 그리고 바로 너 왜 싫어 원해 이제
